In [ ]:
def myFunc(y_true, y_pred, df):
    df[y_true.index.values,'y_pred'] = y_pred
    ground_truths = get_ground_truth(df)
    submission = get_submission(df)
    ap,first_pred_acc = evaluate(submission,ground_truths)
    return ap

In [14]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV, RepeatedKFold
from sklearn.metrics import  make_scorer
import pandas as pd

In [71]:
# X, y = load_iris(return_X_y=True)
# index = ['r%d' % x for x in range(len(y))]
# y_frame = pd.DataFrame(y, index=index)
# sample_weight = np.array([1 + 100 * (i % 25) for i in range(len(X))])
# sample_weight_frame = pd.DataFrame(sample_weight, index=index)
sample_weight_frame = pd.DataFrame({'col1':[1,1/3,1/6,1/10,1/15], 'col2':[1,1,1,1,1]})
sample_weight_frame


,col1,col2
0,1.000000,1
1,0.333333,1
2,0.166667,1
3,0.100000,1
4,0.066667,1


In [73]:
sample_weight_frame.loc[1:2]

,col1,col2
1,0.333333,1
2,0.166667,1


In [70]:
y_true.index.values

array([0, 1, 2, 3, 4])

In [75]:
print()

In [66]:
def myFunc(y_true, y_pred, sample_weight):
    weight = sample_weight.loc[y_true.index.values].values.reshape(-1)
#     print(y_true.index.values)
#     print(weight)
    aa_list = []
    for x, y, z in zip(y_true.values.tolist(), y_pred, weight):
        print((x==y)*z)
        aa_list.append((x==y)*z)
    aa = np.mean(aa_list)
#     print(aa_list)
    return aa

y_true = pd.DataFrame(np.array([True, True, True, True, True]))
y_pred = np.array([False, True, True, True, True])
sample_weight_frame
myFunc(y_true, y_pred, sample_weight_frame)

[0.]
[0.33333333]
[0.16666667]
[0.1]
[0.06666667]


0.13333333333333333

In [49]:
y_true.values.tolist()

[[1], [1], [1], [1], [1]]

In [57]:
list(zip(y_true.values.tolist(), y_pred.values.tolist(), sample_weight_frame.values.reshape(-1).tolist()))

[([1], [0], 1.0),
 ([1], [1], 0.3333333333333333),
 ([1], [1], 0.16666666666666666),
 ([1], [1], 0.1),
 ([1], [1], 0.06666666666666667)]

In [25]:
y_true = [1,1,1,1,1,1,1,1,1,1]
y_pred = [0,0,0,0,0,0,0,0,0,0]
sample_weight2 = sample_weight[0:9]
sample_weight2

array([  1, 101, 201, 301, 401, 501, 601, 701, 801])

In [26]:
y_true.values

AttributeError: 'list' object has no attribute 'values'

In [ ]:
log_loss(y_true.values, y_pred,
                  sample_weight=sample_weight.loc[y_true.index.values].values.reshape(-1),
                  normalize=True)

In [ ]:



X, y = load_iris(return_X_y=True)
index = ['r%d' % x for x in range(len(y))]
y_frame = pd.DataFrame(y, index=index)
sample_weight = np.array([1 + 100 * (i % 25) for i in range(len(X))])
sample_weight_frame = pd.DataFrame(sample_weight, index=index)

# more code

def score_f(y_true, y_pred, sample_weight):
return log_loss(y_true.values, y_pred,
                  sample_weight=sample_weight.loc[y_true.index.values].values.reshape(-1),
                  normalize=True)

score_params = {"sample_weight": sample_weight_frame}

my_scorer = make_scorer(score_f,
                      greater_is_better=False, 
                      needs_proba=True, 
                      needs_threshold=False,
                      **score_params)

grid_clf = GridSearchCV(estimator=rfc,
                      scoring=my_scorer,
                      cv=inner_cv,
                      param_grid=search_params,
                      refit=True,
                      return_train_score=False,
                      iid=False)  # in this usage, the results are the same for `iid=True` and `iid=False`

grid_clf.fit(X, y_frame)

In [67]:
(0+ 1/2 + 2/3)/3

0.38888888888888884

In [5]:
(0*1 + 1/(1+(2)) + 1/3 + 1/4 + 1/5)/(1 + 1/2 + 1/3 + 1/4 + 1/5)

0.5620437956204379

In [6]:
1/(1+(2))

0.6666666666666666

In [8]:
1/(1+2+3)

0.16666666666666666

In [68]:
(0*1 + 1/(1+2) + 1/(1+2+3) )/(1 + 1/3 + 1/6)

0.3333333333333333

In [28]:
y_true = [1,1,1,1,1]
y_pred = [0,1,1,1,1]
y_true == y_pred

False

In [ ]:
from __future__ import division

import numpy as np
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV, RepeatedKFold
from sklearn.metrics import  make_scorer
import pandas as pd

def grid_cv(X_in, y_in, w_in, cv, max_features_grid, use_weighting):
    out_results = dict()

    for k in max_features_grid:
    clf = RandomForestClassifier(n_estimators=256,
                                 criterion="entropy",
                                 warm_start=False,
                                 n_jobs=1,
                                 random_state=RANDOM_STATE,
                                 max_features=k)
    for train_ndx, test_ndx in cv.split(X=X_in, y=y_in):
        X_train = X_in[train_ndx, :]
        y_train = y_in[train_ndx]
        w_train = w_in[train_ndx]
        y_test = y_in[test_ndx]

        clf.fit(X=X_train, y=y_train, sample_weight=w_train)

        y_hat = clf.predict_proba(X=X_in[test_ndx, :])
        if use_weighting:
        w_test = w_in[test_ndx]
        w_i_sum = w_test.sum()
        score = w_i_sum / w_in.sum() * log_loss(y_true=y_test, y_pred=y_hat, sample_weight=w_test)
        else:
        score = log_loss(y_true=y_test, y_pred=y_hat)

        results = out_results.get(k, [])
        results.append(score)
        out_results.update({k: results})

    for k, v in out_results.items():
    if use_weighting:
        mean_score = sum(v)
    else:
        mean_score = np.mean(v)
    out_results.update({k: mean_score})

    best_score = min(out_results.values())
    best_param = min(out_results, key=out_results.get)
    return best_score, best_param


#if __name__ == "__main__":
if True:
    RANDOM_STATE = 1337
    X, y = load_iris(return_X_y=True)
    index = ['r%d' % x for x in range(len(y))]
    y_frame = pd.DataFrame(y, index=index)
    sample_weight = np.array([1 + 100 * (i % 25) for i in range(len(X))])
    sample_weight_frame = pd.DataFrame(sample_weight, index=index)
    # sample_weight = np.array([1 for _ in range(len(X))])

    inner_cv = RepeatedKFold(n_splits=3, n_repeats=1, random_state=RANDOM_STATE)

    outer_cv = RepeatedKFold(n_splits=3, n_repeats=1, random_state=RANDOM_STATE)

    rfc = RandomForestClassifier(n_estimators=256,
                               criterion="entropy",
                               warm_start=False,
                               n_jobs=1,
                               random_state=RANDOM_STATE)
    search_params = {"max_features": [1, 2, 3, 4]}


    def score_f(y_true, y_pred, sample_weight):
        return log_loss(y_true.values, y_pred,
                      sample_weight=sample_weight.loc[y_true.index.values].values.reshape(-1),
                      normalize=True)

    score_params = {"sample_weight": sample_weight_frame}
    my_scorer = make_scorer(score_f,
                          greater_is_better=False, 
                          needs_proba=True, 
                          needs_threshold=False,
                          **score_params)

    grid_clf = GridSearchCV(estimator=rfc,
                          scoring=my_scorer,
                          cv=inner_cv,
                          param_grid=search_params,
                          refit=True,
                          return_train_score=False,
                          iid=False)  # in this usage, the results are the same for `iid=True` and `iid=False`
    grid_clf.fit(X, y_frame)
    print("This is the best out-of-sample score using GridSearchCV: %.6f." % -grid_clf.best_score_)

    msg = """This is the best out-of-sample score %s weighting using grid_cv: %.6f."""
    score_with_weights, param_with_weights = grid_cv(X_in=X,
                                                   y_in=y,
                                                   w_in=sample_weight,
                                                   cv=inner_cv,
                                                   max_features_grid=search_params.get(
                                                     "max_features"),
                                                   use_weighting=True)
    print(msg % ("WITH", score_with_weights))

    score_without_weights, param_without_weights = grid_cv(X_in=X,
                                                         y_in=y,
                                                         w_in=sample_weight,
                                                         cv=inner_cv,
                                                         max_features_grid=search_params.get(
                                                           "max_features"),
                                                         use_weighting=False)
    print(msg % ("WITHOUT", score_without_weights))